# Chemical equilibrium: the basics

Reaktoro performs chemical equilibrium calculations by *minimizing the Gibbs energy of the system*. In a Gibbs energy minimization (GEM) problem, where 
* **temperature and pressure are prescribed** and 
* **the chemical system is closed**. 

Thus, without mass transfer in or out of the system, the initial amounts of chemical elements and electric charge are conserved in the reactive process.

> **Note**: There are other classes of equilibrium problems, where:
>
> * **temperature and/or pressure may be unknown** (e.g., combustion in a rigid and adiabatic chamber where volume and internal energy are specified and temperature and pressure are calculated);
> * **the chemical system is open to certain substances** (e.g., aqueous solution in equilibrium with a mixture of gases with known partial pressures or fugacities, such as the atmosphere).
>
> *Reaktoro's chemical equilibrium algorithm supports all these cases* because it implements an algorithm to solve *parametric Gibbs energy minimization problems with general equilibrium constraints*. Thus, Reaktoro is not restricted to closed systems and prescribed temperature and pressure conditions.

A simple example below demonstrates the *most basic form of performing chemical equilibrium calculations*, modelling the combustion of 1 mol of CH<sub>4</sub> and 0.5 mol of O<sub>2</sub> at 1000 ºC and 100 bar. First, we define a chemical system consisting of only a gaseous phase:

In [1]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

Then, we create a chemical state for this system with conditions given above:

In [2]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

print("=== INITIAL STATE ===")
print(state)

=== INITIAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.0000e+00 |  mol |
| :: O2           | 5.0000e-01 |  mol |
| :: CO2          | 1.0000e-16 |  mol |
| :: CO           | 1.0000e-16 |  mol |
| :: H2O          | 1.0000e-16 |  mol |
| :: H2           | 1.0000e-16 |  mol |
+-----------------+------------+------+


Finally, we create an [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) object to perform the desired chemical equilibrium calculation:

In [3]:
solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 3.7606e-01 |  mol |
| :: O2           | 1.0000e-16 |  mol |
| :: CO2          | 9.3630e-02 |  mol |
| :: CO           | 5.3031e-01 |  mol |
| :: H2O          | 2.8243e-01 |  mol |
| :: H2           | 9.6544e-01 |  mol |
+-----------------+------------+------+


> **Note**: the initial and final amounts of the elements H, C, and O as well as electric charge in the system are identical, e.i., *mass is preserved*.

> **Note**: if you need to perform a series of equilibrium calculations (e.g. in a context of reactive transport simulations), prefer using the [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) class instead of the [equilibrate](https://reaktoro.org/api/namespaceReaktoro.html#a286e2972325304934d631234abef9e87) method, to avoid an overhead of creating an [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) instance at each call of [equilibrate](https://reaktoro.org/api/namespaceReaktoro.html#a286e2972325304934d631234abef9e87).

## Checking if the calculation was successful

Performing chemical equilibrium calculations is not a trivial task. It involves highly complicated algorithms to solve different mathematical problems (e.g., non-linear equations, matrix equations). Therefore, it is possible that a calculation could fail. This can happen due to many factors, among many are
* formulation of the equilibrium problem is ill-formed (e.g. specifying a set of conflicting equilibrium constraints that cannot be achieved chemically/thermodynamically)
* use of poor initial guesses.

Therefore, we recommend to check if the equilibrium calculation was successful by inspecting the [EquilibriumResult](https://reaktoro.org/api/structReaktoro_1_1EquilibriumResult.html) instance that `solver.solve` function returns:

In [4]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

solver = EquilibriumSolver(system)
result = solver.solve(state)

print("Has solver converged?", result.optima.succeeded)

Has solver converged? True


The most **common mistakes** causing the divergence of equilibrium solver:

* the temperature is set in given in celsium, but actuially set with default kelvin,
* the pressure is set in given bar, but actually set in default bar, 
* the chemical system is not including nessesary species that might be forming, 
* to low value of H or O indicating abasence of water, etc.